In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("Bengaluru_House_Data.csv")
print(df.columns)
df.head()

In [ ]:
df.info()
df.isnull().sum()

In [ ]:
df = df.drop("society", axis=1, errors="ignore")
df = df.dropna()
df.isnull().sum()

In [ ]:
df["bhk"] = df["size"].str.split().str[0].astype(int)
df = df.drop("size", axis=1)
df.head()


In [ ]:
def convert_sqft_to_num(x):
    tokens = str(x).split('-')
    if len(tokens) == 2:
        return (float(tokens[0]) + float(tokens[1])) / 2
    try:
        return float(x)
    except:
        return None

df["total_sqft"] = df["total_sqft"].apply(convert_sqft_to_num)
df = df.dropna()
df.head()

In [ ]:
df = df.drop(["availability", "area_type"], axis=1,errors="ignore")
df.head()

In [ ]:
df["price_per_sqft"] = (df["price"] * 100000) / df["total_sqft"]
df.head()

In [ ]:
df = df[df["total_sqft"] / df["bhk"] >= 300]
df.head()

In [ ]:
location_stats = df["location"].value_counts()

location_stats_less_10 = location_stats[location_stats <= 10]

df["location"] = df["location"].apply(
    lambda x: "other" if x in location_stats_less_10 else x
)
df.head()

In [ ]:
df = pd.get_dummies(df, columns=["location"], drop_first=True)
df.head()

In [ ]:
X = df.drop("price", axis=1)
y = df["price"]

In [ ]:
import sys
!"{sys.executable}" -m pip install xgboost

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import joblib

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(
    X,y, test_size=0.2, random_state=42
)

model = XGBRegressor(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train,y_train)

In [ ]:
pred=model.predict(X_test)

In [ ]:
import numpy as np
rmse=np.sqrt(mean_squared_error(y_test, pred))
print("RMSE:", rmse)

In [ ]:
joblib.dump(model, "../backend/model/house_model.pkl")
joblib.dump(X.columns.tolist(),"../backend/model/columns.pkl")
print("Model saved!")